In [6]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import numpy as np
import cv2
import os
import cvlib as cv
import time
import keyboard
# load model
model_gender = load_model('C:/Users/Tan/Desktop/qua_trinh/Gender_model/Gender_Final_.h5')
model_age = load_model('C:/Users/Tan/Desktop/qua_trinh/Age_model/Age_18_45.h5')
model_emotion = load_model('C:/Users/Tan/Desktop/qua_trinh/Emotion_model/Emotion_ver2.h5')
# open webcam
webcam = cv2.VideoCapture(0)
classes_emotion = ['Happy', 'Neural', 'Sad', 'Suprise']
classes_gender = ['Female', 'Male']
# loop through frames
i = 1
j = 0
dot = 0
mean = 0
value = ""
test = [0 for i in range(20)]
while webcam.isOpened():

    # read frame from webcam
    status, frame = webcam.read()

    # apply face detection
    face, confidence = cv.detect_face(frame)

    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY, startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        # # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (150, 150))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        # model.predict return a 2D matrix, ex: [[9.9993384e-01 7.4850512e-05]]
        conf_age = model_age.predict(face_crop)[0]

        # get label with max accuracy
        idx_age = np.argmax(conf_age)
        if i < len(test):
            test[i] = idx_age + 18
            if dot == 0:
                value = ""
                # time.sleep(0.3)
            if dot == 1:
                value = ".."
                # time.sleep(0.3)
            if dot == 2:
                value = "....."
                # time.sleep(0.3)
                dot = 0
            dot = dot + 1

            # print(test[i])
        else:
            mean = np.mean(test)
            value = str(int(mean))
            for k in range(len(test)):
                if k == len(test) -1:
                    break
                test[k] = test[k + 1]
                k = k+1
                # print(k)
            # i = 0
            # print
            i = i-1
            test[i-1] = idx_age + 18
            # print(test[i-1])
        i = i+1

        label_age = "Age: " + value
        label_age = "{}".format(label_age)
        conf_emotion = model_emotion.predict(face_crop)[0]

        # get label with max accuracy
        idx_emotion = np.argmax(conf_emotion)
        # acc = conf[idx] * 100
        # label = classes[idx]
        label_emotion = "Emotion: " + classes_emotion[idx_emotion]
        label_emotion = "{}".format(label_emotion)

        conf_gender = model_gender.predict(face_crop)[0]

        # get label with max accuracy
        idx_gender = np.argmax(conf_gender)
        # acc = conf[idx] * 100
        # label = classes[idx]
        label_gender = "Gender: " + classes_gender[idx_gender]
        # label_gender = "{}: {:.2f}%".format(label_gender, conf_gender[idx_gender] * 100)
        label_gender = "{}".format(label_gender)

        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        # age
        cv2.putText(frame, label_age, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)
        # emotion
        cv2.putText(frame, label_emotion, (startX, Y-40),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)
        cv2.putText(frame, label_gender, (startX, Y-80),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)
    # display output
    cv2.imshow("Age-Emotion-Gender_Detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    if keyboard.is_pressed('s'):  # if key 'q' is pressed
        # print('You Pressed A Key!')
        j = j+1
        cv2.imwrite("C:/Users/Tan/Desktop/qua_trinh/screen_shot/" + str(j)+ ".jpg",frame)
        # break
    if cv2.waitKey(1) & 0xFF == ord('s'):
        j = j+1
        cv2.imwrite("C:/Users/Tan/Desktop/qua_trinh/screen_shot/" + str(j)+ ".jpg",frame)
webcam.release()
cv2.destroyAllWindows()

In [5]:
pip install keyboard


Note: you may need to restart the kernel to use updated packages.
